In [19]:
import os
import urllib, cStringIO

import pymongo as pm

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
from __future__ import division
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64
import datetime
import svg_render_helpers as rdh
import couchdb


In [20]:
auth = pd.read_csv('auth_couch.txt', header = None) 
server_address_input = auth.values[0][0]
couch = couchdb.Server(server_address_input)
db = couch['tsinghua-draw'] 
db.info()

{u'cluster': {u'n': 1, u'q': 8, u'r': 1, u'w': 1},
 u'compact_running': False,
 u'db_name': u'tsinghua-draw',
 u'disk_format_version': 8,
 u'doc_count': 31379,
 u'doc_del_count': 0,
 u'instance_start_time': u'0',
 u'props': {},
 u'purge_seq': u'0-g1AAAAFTeJzLYWBg4MhgTmEQTM4vTc5ISXIwNDLXMwBCwxygFFMeC5BkeACk_gNBViIDQbUHIGrvE6N2AUTtfmLUNkDUzsevNikBSCbVE3RrkgNIXTxhdQogdfYE1SUyJMlDFGUBAD9sXo4',
 u'sizes': {u'active': 45616642, u'external': 51872022, u'file': 66807560},
 u'update_seq': u'31383-g1AAAAFreJzLYWBg4MhgTmEQTM4vTc5ISXIwNDLXMwBCwxygFFMeC5BkeACk_gNBVhIDA78SQeUHIMrvg5Tz_SSofAFE-X6w6eoElTdAlM8HK7-CR3lSApBMqoe5uxafUgeQ0niYUmd8ShVASu1hSufgUZrIkCQPU6eaBQDytGKk'}

In [21]:
img_recs_selector = {'selector': {'dataType':'finalImage', 'colname':'Tsinghua_photodraw_production', 'trialNum':10}, 'limit':1000} # limit defautls to 25, wow
img_recs_temp = db.find(img_recs_selector)
result = map(lambda x: x['subID'], img_recs_temp)
all_sessions_recorded = np.unique(result)
count_sub=0

In [22]:
# post filtering
meta_data = pd.read_csv('final_merged_metadata.csv')
is_THU = meta_data['site']=="THU"
THU_meta = meta_data[is_THU]
valid_sessions = np.unique(THU_meta['session_id'])

sessions_to_render = pd.DataFrame(valid_sessions)
sessions_to_render.columns = ['session_id']
print('we have {} unique kids'.format(len(sessions_to_render)))


we have 132 unique kids


#### Rendering directory

In [23]:
analysis_dir = os.getcwd()
sketch_dir = os.path.join(analysis_dir,'all_tracings_june_2021')

In [24]:
# imrec['category'].split("this ")[1]

## Render out tracings for each subject

In [26]:
reload(rdh)
really_run_this=1
tracing = ['this square', 'this shape']
# canvas_size = 809
# imsize = 400

# canvas_size = 809
canvas_size = 696
imsize = 400

if really_run_this:
   
    for s in sessions_to_render['session_id']:
        
        count_sub = count_sub + 1
        img_recs_selector = {'selector': {'dataType':'finalImage', 'sessionId': s, 'category':{"$in": tracing}}}
        img_recs_temp = db.find(img_recs_selector)
        image_recs = sorted(img_recs_temp, key=lambda image: image['endTrialTime'])
        print 'found {} images for {}'.format(np.size(image_recs), s)
        
        for imrec in image_recs:  
            stroke_recs_selector = {'selector': {'dataType':'stroke','category':imrec['category'], 'sessionId': imrec['sessionId']}}
            stroke_recs_temp = db.find(stroke_recs_selector) 
            sorted_stroke_recs = sorted(stroke_recs_temp, key=lambda stroke: stroke['endStrokeTime'])
            stroke_recs = sorted_stroke_recs
#             print 'found {} strokes'.format(np.size(stroke_recs))
               #  
            category_short = imrec['category'].split("this ")[1]
            category_dir = os.path.join(sketch_dir,category_short)
        
            if not os.path.exists(category_dir):
                os.makedirs(category_dir)
            
            #
            _svg = [] # this keeps track of the strokes from THIS final image
            # in couchdb, stroke_recs had extra info saved in each string, so need to parse actual svg
            for strec in stroke_recs:
                temp_split = strec['svg'].split("d=")
                temp_split_2 = temp_split[1].split("\" fill")
                trimmed_svg = temp_split_2[0]
                _svg.append(trimmed_svg)

                
            
            # don't do adults for now or blank images
            if np.size(stroke_recs) > 0: 
                # render out images
                try:

                    ## now get me some verts and codes!
                    Verts, Codes = rdh.get_verts_and_codes(_svg)
                    outpath = os.path.join(category_dir,'{}_{}.png'.format(imrec['sessionId'],category_short))

                    ## now render out your cumulative sketches and save out as pngs!
                    rdh.render_and_save(Verts,
                                    Codes,
                                    outpath,
                                    line_width=5,
                                    imsize=imsize,
                                    canvas_size=canvas_size)

                except Exception as e:
                    print 'Oops, something went wrong! Here is the error:'  
                    print e
                    pass

found 2 images for Tsinghua_photodraw_production1573778450764
found 2 images for Tsinghua_photodraw_production1573778817054
found 2 images for Tsinghua_photodraw_production1573779019443
found 2 images for Tsinghua_photodraw_production1573779087653
found 2 images for Tsinghua_photodraw_production1573779097000
found 2 images for Tsinghua_photodraw_production1573779630652
found 2 images for Tsinghua_photodraw_production1573779810259
found 2 images for Tsinghua_photodraw_production1573779925870
found 2 images for Tsinghua_photodraw_production1573780074909
found 2 images for Tsinghua_photodraw_production1573780408443
found 2 images for Tsinghua_photodraw_production1573780549952
found 2 images for Tsinghua_photodraw_production1573780632846
found 2 images for Tsinghua_photodraw_production1573781111110
found 2 images for Tsinghua_photodraw_production1573781474552
found 2 images for Tsinghua_photodraw_production1573781508415
found 2 images for Tsinghua_photodraw_production1573781580447
found 2 

In [ ]:
# _svg = []
# for strec in stroke_recs:
#     temp_split = strec['svg'].split("d=")
#     temp_split_2 = temp_split[1].split("\" fill")
#     trimmed_svg = temp_split_2[0]
#     _svg.append(trimmed_svg)

# Verts, Codes = rdh.get_verts_and_codes(_svg)